In [24]:
import pandas as pd
import numpy as np
import matplotlib
from pandas.plotting import table

%matplotlib inline

In [25]:
train = pd.read_csv('/Users/BrianLee/Desktop/input/titanic_train.csv')
test = pd.read_csv('/Users/BrianLee/Desktop/input/titanic_test.csv')

In [26]:
train, test = train[['PassengerId','Survived','Sex','Pclass','Age']], test[['PassengerId','Sex','Pclass','Age']]

In [27]:
titanic_train_test = train.append(test)

titanic_train_test['Age'] = titanic_train_test['Age'].fillna(train['Age'].median())

titanic_Age_range = [(0,10),(11,20),(21,30),(31,50),(51,100)]

Age_range = []
titanic_Age_int = list(map(int,titanic_train_test.Age.values))
for j in range(len(titanic_Age_int)):
    for i in range(len(titanic_Age_range)):
        if titanic_Age_int[j] in range(titanic_Age_range[i][0],titanic_Age_range[i][1]):
            age_range = titanic_Age_range[i]
        else:
            pass
    Age_range.append(age_range)
titanic_train_test['Age_range'] = Age_range

In [28]:
titanic_train_relevant = titanic_train_test[titanic_train_test['PassengerId']<=len(train)]
titanic_test_relevant = titanic_train_test[titanic_train_test['PassengerId']>len(train)]
titanic_train_relevant = titanic_train_relevant.drop(['Age'], axis=1)
titanic_test_relevant = titanic_test_relevant.drop(['Age','Survived'], axis=1)
titanic_train_test_relevant = titanic_train_relevant.append(titanic_test_relevant)
titanic_train_test_relevant = titanic_train_test_relevant.drop('Survived', axis=1)
titanic_train_test_relevant = titanic_train_test_relevant.set_index('PassengerId')


In [29]:
titanic_train_test_relevant = pd.get_dummies(titanic_train_test_relevant, columns=list(titanic_train_test_relevant.columns.values))

In [30]:
titanic_train_final=titanic_train_test_relevant.iloc[0:len(train)]
titanic_test_final=titanic_train_test_relevant.iloc[len(train):len(titanic_train_test_relevant)]

In [31]:
titanic_train_target = titanic_train_relevant['Survived'].astype(int)

In [32]:
titanic_train_final.head()

,"Age_range_(0, 10)","Age_range_(11, 20)","Age_range_(21, 30)","Age_range_(31, 50)","Age_range_(51, 100)",Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
PassengerId,,,,,,,,,,
1,0,0,1,0,0,0,0,1,0,1
2,0,0,0,1,0,1,0,0,1,0
3,0,0,1,0,0,0,0,1,1,0
4,0,0,0,1,0,1,0,0,1,0
5,0,0,0,1,0,0,0,1,0,1


In [33]:
titanic_test_final.head()

,"Age_range_(0, 10)","Age_range_(11, 20)","Age_range_(21, 30)","Age_range_(31, 50)","Age_range_(51, 100)",Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
PassengerId,,,,,,,,,,
892,0,0,0,1,0,0,0,1,0,1
893,0,0,0,1,0,0,0,1,1,0
894,0,0,0,0,1,0,1,0,0,1
895,0,0,1,0,0,0,0,1,0,1
896,0,0,1,0,0,0,0,1,1,0


In [34]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier

In [35]:
gbm = GradientBoostingClassifier()

In [36]:
X_train = titanic_train_final
y_train = titanic_train_target

In [14]:
def find_predicted_values(X_train, y_train,n, ml_algo):
    #n = number of cross validation folds
    #ml_algo = machine learning algorithms
    predictions = []
    kf = KFold(800)#n_splits=n
    
    for train, test in kf.split(X_train,y_train):
        train_data = X_train.iloc[train,:]
        target_data = y_train.iloc[train]
        ml_algo.fit(train_data,target_data)
        test_predict = ml_algo.predict(X_train.iloc[test,:])
        predictions.append(test_predict)
    predictions=np.concatenate(predictions)
    return predictions

In [37]:
gbm_predictions = find_predicted_values(X_train,y_train,4,gbm)

In [38]:
print(metrics.accuracy_score(y_train,gbm_predictions))

0.809203142536


In [39]:
submit_predict = gbm.predict(titanic_test_final).astype(int)

In [40]:
submission = pd.DataFrame({
    "PassengerId": titanic_test_final.index,
    "Survived": submit_predict
    })

submission.to_csv('titanic_gbm_submission.csv',index=False)